# Snowflake Public Data Demo - Tariff Sentiment

This notebook is written to help explore the [Snowflake Public Data Set](https://app.snowflake.com/marketplace/providers/GZTSZAS2KCS/Snowflake%20Public%20Data%20Products), a large catalog of data sets curated by Snowflake to help customers fully unlock their own data. 

This demo specifically uses the SEC Filings data set, and leverages it to analyze the sentiment that different companies may have towards Tariffs.

This notebook will:
- Filter the data for 10-Q forms
- Extract all data that mentions Tariffs
- Pass this data through Cortex to analyze sentiment
- Display graphs and raw results of sentiment analysis

## Download the SEC Snowflake Public Data Set

You can download the SEC Filings data [here](https://app.snowflake.com/marketplace/listing/GZTSZAS2KH9/snowflake-public-data-products-sec-filings?originTab=provider&providerName=Snowflake+Public+Data+Products&profileGlobalName=GZTSZAS2KCS) or run the lines of SQL code below!

Please change the variable "PUBLIC_DATA_DB" to reflect the name you give the database.

If you have not used the marketplace before, or have not verified the email associated with your snowflake login, please follow [these steps](https://docs.snowflake.com/en/user-guide/ui-snowsight-profile#verify-your-email-address)

In [ ]:
-- Request the listing
CALL SYSTEM$REQUEST_LISTING_AND_WAIT('GZTSZAS2KH9');

-- Accept legal terms for the listing. Email verification is required to create the database from listing GZ1MXZFTF1
CALL SYSTEM$ACCEPT_LEGAL_TERMS('DATA_EXCHANGE_LISTING', 'GZTSZAS2KH9');

 -- Download SEC Data
 CREATE DATABASE IF NOT EXISTS MARKETPLACE_CYBERSYN
  FROM LISTING 'GZTSZAS2KH9';

In [ ]:
# Update this to reflect the name of the database where the Snowflake Public Data listing was consumed
PUBLIC_DATA_DB="MARKETPLACE_CYBERSYN"

In [ ]:
-- tag for tracking purposes
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"snowflake_public_data_tariff_sentiment","version":{"major":1, "minor":0},"attributes":{"is_quickstart":0, "source":"sql"}}';

-- Create subset of index table
CREATE TABLE IF NOT EXISTS sec_demo.sec_report_idx AS (
    SELECT
        *
    FROM
        {{PUBLIC_DATA_DB}}.cybersyn.sec_report_index
    WHERE
        form_type = '10-Q'
        AND (
            fiscal_year = 2025
            OR fiscal_year = 2024
        )
);

-- Create subset of report text table
CREATE TABLE IF NOT EXISTS sec_demo.sec_report_text_att AS (
    SELECT
        att.*
    FROM
        {{PUBLIC_DATA_DB}}.cybersyn.sec_report_text_attributes AS att
        INNER JOIN sec_demo.sec_report_idx USING (adsh)
    WHERE
        variable_name = '10-Q Filing Text'
);

## Run SQL statements to extract reports that mention tariffs

In [ ]:
-- Find forms that mention tariffs somewhere
CREATE TABLE IF NOT EXISTS sec_demo.sec_report_mentions_tariff AS (
    WITH mentions_tariff AS (
        SELECT
            DISTINCT adsh
        FROM
            sec_demo.sec_report_text_att
        WHERE
            value ILIKE '%tariff%'
    )
    SELECT
        idx.filed_date,
        text.adsh,
        value AS form_text
    FROM
        mentions_tariff
        INNER JOIN sec_demo.sec_report_text_att AS text USING (adsh)
        INNER JOIN sec_demo.sec_report_idx AS idx USING (adsh)
);

-- Break up forms that mention tariffs into sentences
CREATE TABLE IF NOT EXISTS sec_demo.report_sentences AS (
    SELECT
        adsh,
        seq,
        index,
        value,
    FROM
        sec_demo.sec_report_mentions_tariff,
        LATERAL STRTOK_SPLIT_TO_TABLE(form_text, '.')
);

-- Filter table to only show sentences that mention tariffs
CREATE TABLE IF NOT EXISTS sec_demo.tariff_sentences AS (
    SELECT
        idx.filed_date,
        idx.adsh,
        seq,
        index,
        TRIM(value) AS value
    FROM
        sec_demo.report_sentences
        INNER JOIN sec_demo.sec_report_idx AS idx USING (adsh)
    WHERE
        value ILIKE '%tariff%'
);

-- Combine sentences before/after sentence that mentions tariffs to give more context
CREATE TABLE IF NOT EXISTS sec_demo.tariff_sections AS (
    WITH
        all_sentences AS (
            SELECT DISTINCT
                r.*
            FROM
                sec_demo.tariff_sentences AS t
                LEFT JOIN sec_demo.report_sentences AS r ON (
                    r.seq = t.seq
                    AND r.adsh = t.adsh
                    AND ABS(r.index - t.index) < 3
                )
            ORDER BY
                r.adsh,
                r.seq,
                r.index asc
        )
    SELECT
        adsh,
        LISTAGG(value, '. ') WITHIN GROUP (
            ORDER BY
                seq,
                index asc
        ) AS tariff_section
    FROM
        all_sentences
    GROUP BY
        ALL
);


## Pass report sections to LLM for labelling
This query can take 5-10 minutes to run, it's best to run it ahead of the demo to allow the results to be materialized.

In [ ]:
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'ANY_REGION';

-- Pass tariff sections to Claude to label the sentiment
CREATE TABLE IF NOT EXISTS sec_demo.tariff_section_sentiment AS (
    WITH
        dated_reports AS (
            SELECT
                idx.filed_date,
                sec.*
            FROM
                sec_demo.tariff_sections sec
                INNER JOIN sec_demo.sec_report_idx AS idx USING (adsh)
        )
    SELECT
        *,
        SNOWFLAKE.CORTEX.COMPLETE (
            'claude-4-sonnet',
            CONCAT(
                'This is a snippet of a 10-Q from the SEC, please label if this report is overall Pessimistic, Neutral or Optimistic regarding their business and how it relates to tariffs. We should only label the report as Pessimistic if it explictly calls out tariffs as an increased risk. If tariffs are mentioned in passing or they are part of standard disclaimers then we should label the report as Neutral. Start your reponse with either Pessimistic, Neutral or Optimistic and then breifly explain your reasoning only if the answer is NOT Neutral: ',
                tariff_section
            )
        ) AS sentiment
    FROM
        dated_reports
    WHERE
        LENGTH(tariff_section) < 100000
        AND filed_date > '2025-01-01'
    LIMIT
        1000
);

## Analyze results
This section contains cells to graph and explore the results

In [ ]:
select
    filed_date,
    DATE_TRUNC ('MONTH', filed_date) AS filed_month,
    count(distinct adsh) as tariff_mentions
from
    sec_demo.sec_report_mentions_tariff
    where timediff(DAY, filed_date, current_timestamp())< 150
group by
    all;

In [ ]:
import altair as alt
import pandas as pd
from datetime import datetime
from snowflake.snowpark.functions import col, month, sum, lit

SHOW_VERTICAL_LINE = True


vline_date_and_label = pd.DataFrame({
    'date': [datetime(2025, 4, 2)],
    'label_text': ['April 2nd'], # The text you want to display
    'label_y': [400]    # Y-coordinate for the label
})

vertical_line = alt.Chart(vline_date_and_label).mark_rule(color="red", strokeWidth=2,strokeDash=[5, 5]  ).encode(
    x='date:T'
)
# Text label for the vertical line
text_label = alt.Chart(vline_date_and_label).mark_text(
    align='left',      # Align text to the left of the x-coordinate
    baseline='middle', # Vertically align text to its middle
    dx=7,              # Horizontal offset (pixels) to the right of the line
    dy=-15,            # Vertical offset (pixels) to move it up slightly from label_y if needed
    fontSize=12,
    color='red'
).encode(
    x='date:T',
    y='label_y:Q',     # Use the y-coordinate from our DataFrame
    text='label_text:N' # Use the label text from our DataFrame
)


day_aggregated = group_results.to_df().group_by(col("FILED_DATE")).agg(sum("TARIFF_MENTIONS").alias("TARIFF_MENTIONS"))
filings_chart = alt.Chart(day_aggregated.to_pandas()).mark_bar().encode(
    x= alt.X("FILED_DATE", axis = alt.Axis(labelAngle=0, title='Filed Date')),
    y=alt.Y("TARIFF_MENTIONS",axis = alt.Axis(title='# of Reports'))
).properties(
    width=800,
    height=300,
    title="Number of Filings that Mention Tariffs"  
)



top_chart = filings_chart if not SHOW_VERTICAL_LINE else filings_chart+vertical_line+text_label

top_chart


## Explore company sentiments towards tariffs

In [ ]:
select
    INITCAP(c_idx.company_name) as company_name,
    cik,
    filed_date,
    adsh,
    tariff_section,
    sentiment
from
    sec_demo.tariff_section_sentiment
    inner join sec_demo.sec_report_idx using (adsh)
    inner join {{PUBLIC_DATA_DB}}.cybersyn.company_index as c_idx using (CIK)
where
    sentiment ilike 'Pessimistic%' or sentiment ilike 'Optimistic%';

In [ ]:
import streamlit as st
import re

def bracket_tariff(text):
  def replace_match(match):
    word = match.group(0)
    return f"**:red-background[{word}]**"

  # This regex looks for "tariff" or "tariffs", ignoring case.
  # \b ensures we match whole words only.
  pattern = r"\b(tariff|tariffs)\b"
  return re.sub(pattern, replace_match, text, flags=re.IGNORECASE)


results = graphs_2.to_pandas()
companies = results['COMPANY_NAME']
with st.container(height=1000):

    selected_comp = st.selectbox('Select company', sorted(list(set(companies))))
    
    st.markdown(f'## 10-Q Reports from {selected_comp} that mention tariffs')
    
    
    selected_data = results[results['COMPANY_NAME']==selected_comp]
    selected_data.sort_values(by='FILED_DATE', ascending=False)
    
    for index, row in selected_data.iterrows():
        with st.expander(f"10-Q Filed on {row['FILED_DATE']}"):
            adsh = row['ADSH']
            url = f"https://www.sec.gov/edgar/search/#/q={adsh}"

            llm_raw = row['SENTIMENT'].split(' ')
            prefix = ":red-background[Pessimistic on tariffs]\n\n" if llm_raw[0].startswith('Pessimistic') else  ":green-background[Optimistic on tariffs]\n\n" 
            llm_explaination = ' '.join(llm_raw[1:])
            llm = prefix + llm_explaination
            
            markdown_text = bracket_tariff(row["TARIFF_SECTION"]).replace('$','\$')
            st.markdown(f'**See full report on SEC website**: [{adsh}]({url})')
            st.markdown(f"**LLM Verdict**: {llm}")
            st.markdown('**Section that mentions tariffs**:\n\n'+markdown_text)
